# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from PIL import Image
import os, random, json
from datetime import datetime

import torch
import torch.nn as nn

from torchvision import transforms, datasets, models

In [ ]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
img_size = 224
batch_size = 128
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]
learning_rate = .001

# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {'train': transforms.Compose([transforms.RandomRotation(30),
                                                transforms.RandomResizedCrop(img_size),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ColorJitter(.3, .3, .3),
                                                transforms.ToTensor(),
                                                transforms.Normalize(_mean, _std)]),
                    'valid': transforms.Compose([transforms.Resize(256),
                                                 transforms.CenterCrop(img_size),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(_mean, _std)]),
                    'test': transforms.Compose([transforms.Resize(256),
                                                transforms.CenterCrop(img_size),
                                                transforms.ToTensor(),
                                                transforms.Normalize(_mean, _std)])}

# TODO: Load the datasets with ImageFolder
image_datasets = {key: datasets.ImageFolder(os.path.join(data_dir, key), data_transforms[key]) for key in data_transforms.keys()}

# TODO: Using the image datasets and the transforms, define the dataloaders
dataloaders = {key: torch.utils.data.DataLoader(image_datasets[key], 
                                                batch_size=batch_size,
                                                shuffle=True) for key in data_transforms.keys()}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [ ]:
# Get pre-trained model
model = models.vgg19(pretrained=True)

# Block paramaters
for param in model.parameters():
    param.requires_grad = False

# Get how many input features
input_size = model.classifier[0].in_features

# Define hidden layer sizes
hidd1_size =  int(input_size/8)
hidd2_size =  int(input_size/16)

# Get how many output features
output_size = len(cat_to_name)

# Attach the label mapping, I need it later
model.class_to_idx = image_datasets['train'].class_to_idx

# Create the classifier, this will be attached at the end of the pre-trained network, new layers have requires_grad = True by default
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(input_size, hidd1_size)),
                          ('relu1', nn.ReLU()),
                          ('drop1', nn.Dropout(p=.2)),
                          ('fc2', nn.Linear(hidd1_size, hidd2_size)),
                          ('relu2', nn.ReLU()),
                          ('drop2', nn.Dropout(p=.2)),
                          ('output', nn.Linear(hidd2_size, output_size)),
                          ('softmax', nn.LogSoftmax(dim=1))
                          ]))

# Attach classifier
model.classifier = classifier

In [ ]:
# Set all gradients to 0
model.zero_grad()

# Define criterion and optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam((p for p in model.parameters() if p.requires_grad), lr=learning_rate,)

# Move model to the available device
model = model.to(device)

In [ ]:
epochs = 5
logs = np.zeros(2)
loss_log = []

# Set training mode
for epoch in range(epochs):
    model.train()
    running_loss, accuracy = .0, .0
    
    for X, y in dataloaders['train']:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        y_ = model(X)
        loss = criterion(y_, y)
        loss_log.append(loss.item())
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        print(f"Training batch loss: {loss.item():.3f}", end="\r")
        running_loss += loss.item() 

        _, y_label_ = torch.max(torch.exp(y_), 1)
        accuracy += (y_label_ == y).sum().item()
        
    train_loss = running_loss / len(dataloaders['train'].dataset)
    train_acc = accuracy / len(dataloaders['train'].dataset)
    print(f"Epoch: {epoch+1}/{epochs}", f"Train Loss: {train_loss:.3f}", f"Train Accuracy: {train_acc:.3f}")
    
    # At the end of every epoch put model in evaluation mode
    model.eval()
    running_loss, accuracy = .0, .0

    # Evaluate
    with torch.no_grad():
        for X, y in dataloaders['valid']:
            X, y = X.to(device), y.to(device)
            y_ = model(X)
            
            loss = criterion(y_, y)
            running_loss += loss.item()
            
            _, y_label_ = torch.max(torch.exp(y_), 1)
            accuracy += (y_label_ == y).sum().item()
        
        valid_loss = running_loss / len(dataloaders['valid'].dataset)
        valid_acc = accuracy / len(dataloaders['valid'].dataset)
        print(f"Epoch: {epoch+1}/{epochs}", f"Validation Loss: {valid_loss:.3f}", f"Validation Accuracy: {valid_acc:.3f}")

In [ ]:
# Visualise the loss
plt.plot(loss_log)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
with torch.no_grad():
    accuracy = .0
    for X, y in dataloaders['test']:
        X, y = X.to(device), y.to(device)
        y_ = model(X)

        _, y_label_ = torch.max(torch.exp(y_), 1)
        accuracy += (y_label_ == y).sum().item()

    test_acc = accuracy / len(dataloaders['test'].dataset)
    print(f"Testing Accuracy: {test_acc:.3f}")

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint
curr_timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
checkpoint = {'classifier' : classifier,
              'batch_size': batch_size,
              'learning_rate': learning_rate,
              'epochs': epochs,
              'model_name': 'vgg19',
              'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict(),
              'class_to_idx': model.class_to_idx}

torch.save(checkpoint, f'checkpoint-{curr_timestamp}.pth')

In [ ]:
savedir = 'checkpoint'
if not os.path.exists(savedir):
    print(f"{savedir} doesn't exist, creating it")
    os.makedirs(savedir)

# Save checkpoint in save directory
file_path = f'{savedir}/{arch}_checkpoint_{curr_timestamp}.pth'
print(file_path)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath='checkpoint.pth'):
    checkpoint = torch.load(filepath)
    model = getattr(models, checkpoint['model_name'])(pretrained=True)
    model.classifier = checkpoint['classifier']
    model.epochs = checkpoint['epochs']
    model.optimizer = checkpoint['optimizer']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image_old(image):
    # Note for the reviewer: this is the function I previously submitted
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Using the same pre-processing done earlier
    img_size = 224
    _mean = [0.485, 0.456, 0.406]
    _std = [0.229, 0.224, 0.225]
    transformations = transforms.Compose([transforms.Resize(256),
                                          transforms.CenterCrop(img_size),
                                          transforms.ToTensor(),
                                          transforms.Normalize(_mean, _std)])
    
    return transformations(Image.open(image))

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    img_size = 224
    _mean = [0.485, 0.456, 0.406]
    _std = [0.229, 0.224, 0.225]

    image = Image.open(select_random_pic('valid'))

    # Find ratio between larger and smaller side
    ratio = image.size[1] / image.size[0]

    # Resize smaller side to 256 and larger side to 256* ratio
    image = image.resize((img_size, int(ratio*256)))

    # Crop image so that it's centered and 224x224
    image = image.crop((0,0,224,224))

    # Convert to numpy array
    np_image = np.array(image)/255

    # Normalize
    norm_image = (np_image - _mean) / _std
    norm_image = norm_image.transpose((2, 0, 1))
    
    return torch.from_numpy(norm_image)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
# Small helper function, given a set returns a different photo each time 
def select_random_pic(picture_set):
    rand_class = random.choice(os.listdir(f'{data_dir}/{picture_set}/'))
    rand_pic = random.choice(os.listdir(f'{data_dir}/{picture_set}/{rand_class}'))
    return f'{data_dir}/{picture_set}/{rand_class}/{rand_pic}'

In [ ]:
# Checking if everything works
img = process_image(select_random_pic('valid'))

# imshow returns Type error when fed with tensor when running pytorch 0.4.0 on Workspace 
# I have to convert it first to a Numpy array
# Error doesn't prompt on my local machine with Torch 1.0.0
if torch.__version__ == '1.0.0': 
    imshow(img)
else:
    imshow(img.numpy())

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    model = model.to(device)
    
    X = process_image(image_path).unsqueeze(0)
    y_ = model(X.to(device))
    
    top_prob, top_indexes = torch.exp(y_).data.topk(topk)
    
    # The classifier returns indexes, I need to convert them back to classes
    idx_to_class = {model.class_to_idx[k]: k for k in model.class_to_idx}
    top_classes = [idx_to_class[x] for x in top_indexes.cpu().numpy().flatten()]
    
    return(top_prob.cpu().numpy().flatten(), top_classes)

In [ ]:
# Quick check
image_path = select_random_pic('valid')
model = load_checkpoint('checkpoint.pth')
topk_prob, topk_index = predict(image, model)

print(topk_prob)
print(topk_index)
print(image_path)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
image_path = select_random_pic('valid')
model = load_checkpoint('checkpoint.pth')
topk_prob, topk_index = predict(image_path, model)

fig, (ax1, ax2) = plt.subplots(figsize=(6,12), nrows=2)

titles = [cat_to_name[x] for x in topk_index]
actual_lbl = cat_to_name[image_path.split('/')[2]]
predic_lbl = titles[np.argmax(topk_prob)]

ax1.set_title(f'Actual label: {actual_lbl}')
ax1.imshow(Image.open(image))
ax1.axis('off')

ax2.set_title(f'Predicted label: {predic_lbl}')
ax2.set_yticks(np.arange(5))
ax2.set_yticklabels(titles)
ax2.invert_yaxis()
ax2.set_xlabel('Probability')
ax2.barh(yp, topk_prob, align='center')

plt.show()